# Qiskit Runtime REST API query & job submission [Sampler Primitive]

## (Optional) Get temporary Access token from Auth API via API Token. 

This is especially useful if you would like control over tokens, such as token invalidation. Alternatively, you can work directly with your IBM Quantum Platform API token.

More details can be found in ["Setup an IBM Quantum Channel" Guide](https://docs.quantum.ibm.com/guides/setup-channel#set-up-to-use-ibm-quantum-platform-with-rest-api) in the IBM Quantum Documentation. 

In [1]:
import requests

with open('token') as file:
    token=file.read()

url = 'https://auth.quantum-computing.ibm.com/api/users/loginWithToken'
input={'apiToken': token}
auth_response = requests.post(url, json=input)
auth_id=auth_response.json()['id']

## GET available backends

In [2]:
url_backends = 'https://api.quantum-computing.ibm.com/runtime/backends'
headers = {'Content-Type': 'application/json',
            'x-access-token':auth_id}

backends_response = requests.get(url_backends, headers=headers)

print(backends_response.json()['devices'][:5],"...")

['ibm_quebec', 'ibm_pinguino1', 'ibm_pinguino2', 'ibm_kyoto', 'alt_kawasaki'] ...


## Run a job

### set up qasm circuit

In [3]:
qasm_string='''
OPENQASM 3;
include "stdgates.inc";
qreg q[2];
creg c[2];
x q[0];
cx q[0], q[1];
c[0] = measure q[0]; 
c[1] = measure q[1];
'''

In [179]:
from qiskit.circuit.random import random_circuit
from qiskit import QuantumCircuit, qasm2, qasm3
 
random_circ = random_circuit(5, depth=3, seed=42).decompose(reps=3)
qasm_string = qasm2.dumps(random_circ)

In [4]:
qasm_string

'\nOPENQASM 3;\ninclude "stdgates.inc";\nqreg q[2];\ncreg c[2];\nx q[0];\ncx q[0], q[1];\nc[0] = measure q[0]; \nc[1] = measure q[1];\n'

## Transpile circuit using Qiskit Transpiler REST API

Query the cloud Transpilation service at https://cloud-transpiler.quantum-computing.ibm.com/docs . See more details in the IBM Quantum Documentation guides on ["Transpilation using REST API"](https://docs.quantum.ibm.com/guides/transpile-rest-api). Note that [transpilation is now necessary](https://docs.quantum.ibm.com/announcements/product-updates/2024-02-14-qiskit-runtime-primitives-update) prior to submitting a circuit to IBM Quantum backends. 

In [5]:
backend='ibm_brisbane'

In [6]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json",
        }
body= {
    "qasm_circuits": qasm_string,
}

params = {
    "backend": backend,
    "use_ai": True,
    "optimization_level": 1,
    "coupling_map": [],
    "qiskit_transpile_options": [],
    "ai_layout_mode": 'OPTIMIZE', # 'KEEP', 'OPTIMIZE', 'IMPROVE',
    }



resp = requests.post(
            "https://cloud-transpiler.quantum.ibm.com/transpile",
            headers=headers,
            json=body,
            params = params,
        )


In [ ]:
task_id=resp.json()['task_id']
task_id

### Request the transpilation service results using the task_id

In [ ]:
res = requests.get(url=f"https://cloud-transpiler.quantum.ibm.com/transpile/{task_id}", headers=headers)
print(res.json())

In [11]:
if res.json().get("state") == "SUCCESS":
    resulting_qasm=res.json().get("result")[0].get("qasm")

print(resulting_qasm[:112])

OPENQASM 3.0; include "stdgates.inc"; gate rzx_140040114706704(_gate_p_0) _gate_q_0, _gate_q_1 {   h _gate_q_1; 


## Run Circuit using Runtime REST API

Note the jobs below use [Qiskit Runtime V2 primitives](https://docs.quantum.ibm.com/api/migration-guides/v2-primitives).  

Both SamplerV2 and EstimatorV2 take one or more primitive unified blocs (PUBs) as the input. Each PUB is a tuple that contains one circuit and the data broadcasted to that circuit, which can be multiple observables and parameters. Each PUB returns a result. More details on using the Runtime REST API can be found in the IBM Quantum Documentation guide on ["Primitives with REST API"](https://docs.quantum.ibm.com/guides/primitives-rest-api).

In [ ]:
url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
hub = '<YOUR HUB>'
group = '<YOUR GROUP>'
project = '<YOUR PROJECT>'

headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }
job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "hub": hub,
    "group": group,
    "project": project,
    "start_session": "False", #set to False if you just need to run a single job.  
    "params": {
        "pubs": [[resulting_qasm],[resulting_qasm,None,500]], #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "version": 2 #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
}}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

### check job status

In [196]:
response_status_singlejob= requests.get(url+'/'+job_id, headers=headers)
response_status_singlejob.json().get('state')

{'status': 'Completed', 'reason': '', 'reasonCode': None}

### Get job results

In [144]:
response_result= requests.get(url+'/'+job_id+'/results', headers=headers)

res_dict=response_result.json()

counts=res_dict['results'][0]['data']['c']['samples']
print(counts[:20])

['0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1', '0x1']


#### Convert counts from hexadecimal format

In [112]:
countslist=[int(x,base=16) for x in res_dict['results'][1]['data']['c']['samples']]
print(countslist[:20])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [113]:
from collections import Counter
count_counts=Counter(countslist)
count_counts

Counter({1: 490, 0: 7, 3: 3})

## Exploring Qiskit Runtime Options

In [ ]:
import requests
 
url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
  'Content-Type': 'application/json',
  'x-access-token':auth_id,
  'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' # specifying the application you might be running from. For an actual Integration project, this option is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
  }
job_input = {
  'program_id': 'sampler',
  "backend": backend, 
  "hub": hub,
  "group": group,
  "project": project,
  "start_session": "False", # set to False if you just need to run a single job.  
  "params": {
      "pubs": [[resulting_qasm]], # primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
      "version": 2, # this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
      "options": {
          "dynamical_decoupling": { 
              "enable": True,
              "sequence_type": 'XpXm',
              "extra_slack_distribution": 'middle',
              "scheduling_method": 'alap',
          },
          #"twirling": { 
          #    "enable_gates": True, 
          #    "enable_measure": True,
          #    "num_randomizations": "auto",
          #    "shots_per_randomization": "auto",
          #    "strategy": "active-accum",
          #    },
              
      },
  }
}
 
response = requests.post(url, headers=headers, json=job_input)
 
if response.status_code == 200:
  job_id = response.json().get('id')
  print("Job created:",response.text)
else:
  print(f"Error: {response.status_code}")

## Execution Modes using REST API

You can run your Qiskit primitive workloads using REST APIs in one of three execution modes, depending on your needs: job, session, and batch. More details on specific modes can be found in the IBM Quantum Documentation guide on ["Execution Modes using REST API"](https://docs.quantum.ibm.com/guides/execution-modes-rest-api). 

### Session Mode

A session is a Qiskit Runtime feature that lets you efficiently run multi-job iterative workloads on quantum computers. 

#### Start a Session

Begin with creating a session and obtaining a `sessionId`.

In [ ]:
import json
 
sessionsUrl = "https://api.quantum-computing.ibm.com/runtime/sessions"
 
headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
  "Content-Type": "application/json" 
}
 
instance_string = hub+'/'+group+'/'+project
payload = {
  "backend": backend,
  "instance": instance_string,
}
 
response = requests.request("POST", sessionsUrl, json=payload,  headers=headersList)
 
sessionId = response.json()['id']
 
print(response.json())

#### Submit job(s) against resulting Session ID

In [ ]:
import requests

url = 'https://api.quantum-computing.ibm.com/runtime/jobs'
headers = {
    'Content-Type': 'application/json',
    'x-access-token':auth_id,
    'x-qx-client-application': 'qiskit-version-2/0.39.2/'+'your_application' #specifying the application you might be running from. For an actual Integration project, this option it is invaluable to know where jobs are coming from. At this time the "qiskit-version-2/0.39.2/" string is a necessary prefix.
    }

job_input = {
    'program_id': 'sampler',
    "backend": backend, 
    "hub": hub,
    "group": group,
    "project": project,
    "session_id": sessionId, # This specifies the previously created Session 
    "params": {
        "pubs": [[resulting_qasm],[resulting_qasm,None,100]], #primitive unified blocs (PUBs) containing one circuit each. c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "version": 2, #this defines the version of the Qiskit Runtime Primitive to use, c.f. https://docs.quantum.ibm.com/api/migration-guides/v2-primitives
        "options":{
                "transpilation":{"optimization_level": 1},
                "twirling": {"enable_gates": True,"enable_measure": True}, #c.f. documentation at https://docs.quantum.ibm.com/run/configure-error-mitigation#custom-error-settings-v2-primitives 
                # "dynamical_decoupling": {"enable": True, "sequence_type": "XpXm"},                        
                    },

}
        # "circuits": [resulting_qasm] #only use this input for Primitives V1, i.e. "version": 1
}

response = requests.post(url, headers=headers, json=job_input)

if response.status_code == 200:
    job_id_sampler = response.json().get('id')
    print("Job created:",response.text)
else:
    print(f"Error: {response.status_code}")

#### Get the Job Results

In [60]:
response_status= requests.get(url+'/'+job_id_sampler, headers=headers)
response_status.json().get('state')

{'status': 'Completed', 'reason': '', 'reasonCode': None}

In [61]:
response_sampler = requests.get(url+'/'+job_id_sampler+'/results', headers=headers)
res_dict_sampler=response_sampler.json()

counts=res_dict_sampler['results'][0]['data']['c']['samples']
print(counts[:20])

['0x1', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1', '0x1', '0x0', '0x0', '0x0', '0x0', '0x0', '0x1', '0x0', '0x0', '0x1', '0x0', '0x0']


### Batch Mode

Alternatively, you can submit a batch job by specifying the mode in the request payload. Batch mode can help shorten processing time if all jobs can be provided at the outset. More details about batch mode can be found in the documentation.

In [ ]:
import json
 
sessionsUrl = "https://api.quantum-computing.ibm.com/runtime/sessions"
 
headersList = {
  "Accept": "application/json",
  'x-access-token':auth_id,
  "Content-Type": "application/json" 
}
 
instance_string = hub+'/'+group+'/'+project

payload = json.dumps({
  "backend": backend,
  "instance": instance_string,
  "mode": "batch"
})
 
response = requests.request("POST", sessionsUrl, data=payload,  headers=headersList)
 
sessionId = response.json()['id']

#### Close Session

It is very good practice to close a Session when all jobs are done. This will reduce wait time for subsequent Users. More details can be found in the [documentation](https://docs.quantum.ibm.com/run/run-jobs-in-session).

In [87]:
closureURL="https://api.quantum-computing.ibm.com/runtime/sessions/"+sessionId+"/close"

headersList = {
  "Accept": "application/json",
  # "Authorization": "Bearer "+token,
  'x-access-token':auth_id,
}
 
closure_response = requests.request(
    "DELETE",
    closureURL, 
    headers=headersList)

print("Session closure response ok?:",closure_response.ok,closure_response.text)

Session closure response ok?: True 
